# COVID-19 Hospitalization Challenge

Check the README.md file for challange details.

In [36]:
import pandas as pd
import numpy as np
from matplotlib.dates import DateFormatter
import json
import matplotlib.pyplot as plt
import os, sys
import seaborn as sns
import random
from datetime import datetime, date
from scipy import stats


plt.style.use('seaborn') # ggplot
pd.options.display.precision = 2
pd.options.display.max_rows = 100
pd.options.display.max_columns = None ### watch out


In [37]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


# Load data and merge files

In [38]:
filenames = [
    '../data/original_data/Canada_Hosp1_COVID_InpatientData.xlsx',
    '../data/original_data/Canada_Hosp2_COVID_InpatientData.xlsx'
]

In [50]:
df_list = []

for file in filenames:
    tmp1 = pd.read_excel(file, sheet_name=0)
    tmp2 = pd.read_excel(file, sheet_name=2)
    df1 = tmp1.merge(tmp2.drop(columns=['id']), left_on='id', right_on='parent_id', how='inner')
    df_list.append(df1)

df = pd.concat(df_list)

print(df.shape)
display(df.head(3))

(707, 65)


,id,reason_for_admission,age,sex,ethnicity,ethnicity_other,height,weight,comorbidities,comorbidities_other,received_covid_vaccine,covid_vaccine,medications,smoking_history,year_they_quit,previous_er_visit_within_14_days,admission_disposition,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,motor,verbal,eye,intubated,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,aptt_aptr,pt,alt,ast,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,pao2,pao2_fio2,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,parent_id,did_the_patient_expire_in_hospital,reason_for_death,days_in_hospital_prior_to_expiration,hospital_length_of_stay,icu_length_of_stay,days_in_hospital_prior_to_icu_admission,time_on_mechanical_ventilation,days_in_hospital_prior_to_mechanical_ventilation,days_to_first_covid19_test_negative
0,1,Fever [R50.9],74,Male,"""[]""",NaN,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""","GERD (gastroesophageal reflux disease), Dement...",NaN,NaN,"""[{\""medications\"":\""TYLENOL EXTRA STRENGTH\"",...",NaN,NaN,No,WARD,99.0,62.0,85.0,20.0,93.0,37.5,6.0,4.0,4.0,No,11.0,5.63,132.0,0.41,72.1,23.4,325.0,18.5,128.0,44,14.1,53.0,49.0,165.0,135.0,3.9,10.0,2.3,NaN,NaN,NaN,NaN,NaN,1.1,NaN,NaN,223.0,NaN,1,Yes,Hospital acquired Pneumonia,20.0,21,NaN,NaN,NaN,NaN,NaN
1,2,Pneumonia [J18.9],61,Female,"""[]""",NaN,154.9,73.6,"""[\""Hypertension\"",\""Other\""]""","Closed fracture of fifth metatarsal bone, Dive...",NaN,NaN,"""[{\""medications\"":\""CANDESARTAN\"",\""dosage\"":...",NaN,NaN,No,WARD,122.0,77.0,93.0,36.0,88.0,38.0,6.0,5.0,4.0,No,5.0,4.49,121.0,0.35,78.6,26.9,343.0,14.2,236.0,28,12.9,68.0,68.0,49.0,137.0,3.8,12.0,1.5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2354,95.6,NaN,2,No,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
2,3,Pneumonia [J18.9],58,Female,"""[]""",NaN,NaN,NaN,"""[\""Hypertension\""]""",NaN,NaN,NaN,"""[{\""medications\"":\""\"",\""dosage\"":\""\"",\""freq...",NaN,NaN,No,WARD,109.0,72.0,73.0,24.0,91.0,38.3,6.0,5.0,3.0,No,6.2,4.31,119.0,0.36,84.0,27.6,329.0,12.5,249.0,NaN,NaN,52.0,76.0,71.0,135.0,3.7,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,No,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN


## Clean up and simplify the features

In [51]:
df = df.rename(columns={'did_the_patient_expire_in_hospital': 'did_the_patient_die_in_hospital',
                       'days_in_hospital_prior_to_expiration': 'days_in_hospital_prior_to_death'})
df['smoking_history'].replace('Smoker:.+', value='Smoker', regex=True, inplace=True)

def to_list(value):
    """Formated as nested json"""
    return json.loads(json.loads(value))


df['comorbidities'] = df['comorbidities'].apply(to_list)
print(df.shape)
df.head()

(707, 65)


,id,reason_for_admission,age,sex,ethnicity,ethnicity_other,height,weight,comorbidities,comorbidities_other,received_covid_vaccine,covid_vaccine,medications,smoking_history,year_they_quit,previous_er_visit_within_14_days,admission_disposition,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,motor,verbal,eye,intubated,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,aptt_aptr,pt,alt,ast,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,pao2,pao2_fio2,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,parent_id,did_the_patient_die_in_hospital,reason_for_death,days_in_hospital_prior_to_death,hospital_length_of_stay,icu_length_of_stay,days_in_hospital_prior_to_icu_admission,time_on_mechanical_ventilation,days_in_hospital_prior_to_mechanical_ventilation,days_to_first_covid19_test_negative
0,1,Fever [R50.9],74,Male,"""[]""",NaN,NaN,NaN,"[Hypertension, Diabetes, Other]","GERD (gastroesophageal reflux disease), Dement...",NaN,NaN,"""[{\""medications\"":\""TYLENOL EXTRA STRENGTH\"",...",NaN,NaN,No,WARD,99.0,62.0,85.0,20.0,93.0,37.5,6.0,4.0,4.0,No,11.0,5.63,132.0,0.41,72.1,23.4,325.0,18.5,128.0,44,14.1,53.0,49.0,165.0,135.0,3.9,10.0,2.3,NaN,NaN,NaN,NaN,NaN,1.1,NaN,NaN,223.0,NaN,1,Yes,Hospital acquired Pneumonia,20.0,21,NaN,NaN,NaN,NaN,NaN
1,2,Pneumonia [J18.9],61,Female,"""[]""",NaN,154.9,73.6,"[Hypertension, Other]","Closed fracture of fifth metatarsal bone, Dive...",NaN,NaN,"""[{\""medications\"":\""CANDESARTAN\"",\""dosage\"":...",NaN,NaN,No,WARD,122.0,77.0,93.0,36.0,88.0,38.0,6.0,5.0,4.0,No,5.0,4.49,121.0,0.35,78.6,26.9,343.0,14.2,236.0,28,12.9,68.0,68.0,49.0,137.0,3.8,12.0,1.5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2354,95.6,NaN,2,No,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
2,3,Pneumonia [J18.9],58,Female,"""[]""",NaN,NaN,NaN,[Hypertension],NaN,NaN,NaN,"""[{\""medications\"":\""\"",\""dosage\"":\""\"",\""freq...",NaN,NaN,No,WARD,109.0,72.0,73.0,24.0,91.0,38.3,6.0,5.0,3.0,No,6.2,4.31,119.0,0.36,84.0,27.6,329.0,12.5,249.0,NaN,NaN,52.0,76.0,71.0,135.0,3.7,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,No,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN
3,4,Suspected COVID-19 virus infection [U07.2],94,Male,"""[]""",NaN,182.8,66.2,"[Hypertension, Other]","Parkinson's Disease, Back pain, Benign Prostat...",NaN,NaN,"""[{\""medications\"":\""ACETYLSALICYLIC ACID TAB...",NaN,NaN,No,WARD,165.0,71.0,92.0,16.0,96.0,37.5,6.0,5.0,4.0,No,5.2,4.18,137.0,0.43,102.4,32.8,320.0,17.2,98.0,NaN,NaN,6.0,24.0,146.0,135.0,2.8,10.0,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Yes,COVID-19 Pneumonia,9.0,9,NaN,NaN,NaN,NaN,NaN
4,5,"Febrile respiratory illness [J98.9, R50.9]",91,Male,"""[]""",NaN,NaN,NaN,"[Chronic cardiac disease (not hypertension), H...","TAVI, Atrial fibrillation, Gastric Reflux, Pac...",NaN,NaN,"""[{\""medications\"":\""TRAZODONE\"",\""dosage\"":\""...",Ex-smoker,NaN,No,WARD,131.0,76.0,60.0,18.0,95.0,38.2,6.0,5.0,4.0,No,4.4,4.65,127.0,0.41,88.4,27.3,309.0,17.6,74.0,NaN,NaN,17.0,26.0,109.0,142.0,3.8,19.0,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Yes,COVID-Pneumonia,9.0,9,NaN,NaN,NaN,NaN,NaN


In [52]:
# To drop
# too much null values, harder to impute
always_null = ['high_senstivity_cardiac_troponin', 'hs_crp', 'received_covid_vaccine', 'covid_vaccine', 'ethnicity', 
            'ethnicity_other', 'esr', 'ferritin', 'pao2', 'pao2_fio2', 'ph']
ids = ['parent_id', 'id', 'days_in_hospital_prior_to_death']
# reduce complexity, those columns needs time to process correctly
exclude = ['medications', 'comorbidities_other', 'year_they_quit', 'reason_for_admission', 
           'days_to_first_covid19_test_negative', 'reason_for_death']


# will drop for the sake of time, but possible to impute
high_null = ['crp', 'd_dimer', 'inr', 'aptt_aptr']

# will keep those, need to be handled
empty_is_zero = ['reason_for_death', 'days_in_hospital_prior_to_death',  #?
                 'icu_length_of_stay', 'time_on_mechanical_ventilation']
empty_is_infinity = ['days_to_first_covid19_test_negative'] #?


df = df.drop(columns=always_null+ids+high_null+exclude)

# missing values in response, drop them
print('Missing values in response: ', df['did_the_patient_die_in_hospital'].isna().sum())
df = df.dropna(subset='did_the_patient_die_in_hospital')


print(df.shape)
df.head()

Missing values in response:  6
(701, 41)


,age,sex,height,weight,comorbidities,smoking_history,previous_er_visit_within_14_days,admission_disposition,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,motor,verbal,eye,intubated,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,pt,alt,ast,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,did_the_patient_die_in_hospital,hospital_length_of_stay,icu_length_of_stay,days_in_hospital_prior_to_icu_admission,time_on_mechanical_ventilation,days_in_hospital_prior_to_mechanical_ventilation
0,74,Male,NaN,NaN,"[Hypertension, Diabetes, Other]",NaN,No,WARD,99.0,62.0,85.0,20.0,93.0,37.5,6.0,4.0,4.0,No,11.0,5.63,132.0,0.41,72.1,23.4,325.0,18.5,128.0,14.1,53.0,49.0,165.0,135.0,3.9,10.0,2.3,Yes,21,NaN,NaN,NaN,NaN
1,61,Female,154.9,73.6,"[Hypertension, Other]",NaN,No,WARD,122.0,77.0,93.0,36.0,88.0,38.0,6.0,5.0,4.0,No,5.0,4.49,121.0,0.35,78.6,26.9,343.0,14.2,236.0,12.9,68.0,68.0,49.0,137.0,3.8,12.0,1.5,No,5,NaN,NaN,NaN,NaN
2,58,Female,NaN,NaN,[Hypertension],NaN,No,WARD,109.0,72.0,73.0,24.0,91.0,38.3,6.0,5.0,3.0,No,6.2,4.31,119.0,0.36,84.0,27.6,329.0,12.5,249.0,NaN,52.0,76.0,71.0,135.0,3.7,7.0,1.0,No,7,NaN,NaN,NaN,NaN
3,94,Male,182.8,66.2,"[Hypertension, Other]",NaN,No,WARD,165.0,71.0,92.0,16.0,96.0,37.5,6.0,5.0,4.0,No,5.2,4.18,137.0,0.43,102.4,32.8,320.0,17.2,98.0,NaN,6.0,24.0,146.0,135.0,2.8,10.0,4.5,Yes,9,NaN,NaN,NaN,NaN
4,91,Male,NaN,NaN,"[Chronic cardiac disease (not hypertension), H...",Ex-smoker,No,WARD,131.0,76.0,60.0,18.0,95.0,38.2,6.0,5.0,4.0,No,4.4,4.65,127.0,0.41,88.4,27.3,309.0,17.6,74.0,NaN,17.0,26.0,109.0,142.0,3.8,19.0,3.2,Yes,9,NaN,NaN,NaN,NaN


In [53]:
df.to_csv('../data/challenge_data/covid_hospitalization_dataset.csv', index=False)